# **Solution Overview: Team Detection and Classification with YOLOv4**

The goal of this project is to provide sample code and a proposal running for a team detection and classification system that can distinguish among players of different teams in a given image or video. The solution aims to automatically identify and locate players on the field and assign the appropriate team labels to each player.

YOLO (You Only Look Once) is a popular real-time object detection algorithm that directly predicts bounding boxes and class probabilities in a single pass through the network. It is a one stage object detector that divides the input image into a grid and applies convolutional layers to simultaneously predict bounding boxes and their associated class probabilities.

 ## **Dataset Preparation**

In order to use the YOLO algorithm for this problem, we would need to prepare the dataset in a specific format suitable for YOLO's training requirements.

YOLO requires a specific label format and annotations of the images. We can annotate the training images by drawing bounding boxes around the players and labeling them with their corresponding team classes (for example, Team 1 and Team 2), by using annotation tools like LabelImg. The annotations should be saved in a format that includes the bounding box coordinates and class labels for each player, as *.txt file per image (if no objects in image, no *.txt file is required).

Finally, the annotated dataset is divided into training, validation, and testing sets. Typically, a common split is around 70-80% for training, 10-15% for validation, and the remaining portion for testing.

## **Organize Directories and Configuration Files**

This step involves properly configuring the custom .cfg, obj.data, obj.names, train.txt and test.txt files necessary for YOLO. These files are customized based on the problem, the dataset and based on specific requirements.  Additional information and comments within the implementation code provide  details on the customization.

## **Model Development**

The YOLOv4 architecture is utilized for team detection and classification. Pre-trained weights for the convolutional layers are downloaded, enhancing the accuracy of the custom object detector without requiring extensive training.

The network is trained using the annotated dataset and the customized YOLOv4 configuration file. The training is performed using the Darknet framework, optimizing the model parameters to minimize the classification error. The training progress, including loss and accuracy, can be visualized using the chart.png file.

## **Inference and Deployment**
The Mean Average Precision (mAP) of the trained model is computed to assess its performance on the test set. The yolov4-obj_best.weights file, representing the best-performing weights achieved during training, is used for the evaluation.

The output predictions, including bounding boxes and class labels, can be displayed or saved for further analysis or visualization. The IoU, mAP, precision, and recall metrics provide a quantitative assessment of the model's performance in accurately detecting and classifying teams and players.


# Cloning and Building Darknet
The following cells will clone darknet from AlexeyAB's repository, adjust the Makefile to enable OPENCV and GPU for darknet and then build darknet.

In [ ]:
# Clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# Change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# Verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# make darknet (builds darknet so that you can use the darknet executable file)
!make

# Defining Helper Functions

These functions are helper functions that will allow us to show images in  Colab Notebook after running detections, as well as download images.

In [ ]:
# Helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to download a file
def download(path):
  from google.colab import files
  files.download(path)

# Upload the files to use
Upload files from Google Drive or from your local machine to the cloud VM and run detections.

For example, I'm using my Google Drive where I could have stored the dataset.



In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

# symbolic link so that the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

# cd back into the darknet folder
%cd darknet

# this is where my datasets are stored within my Google Drive
!ls /mydrive/yolov4

# copy over both datasets into the root directory of the Colab VM
!cp /mydrive/yolov4/obj.zip ../
!cp /mydrive/yolov4/test.zip ../

In [ ]:
# unzip the datasets and their contents so that they are now in /darknet/data/ folder
!unzip ../obj.zip -d data/
!unzip ../test.zip -d data/

# Configuring Files for Training
This step involves properly configuring the custom .cfg, obj.data, obj.names, train.txt and test.txt files necessary for YOLO.

## i) Cfg File

This file has to be customized depending on the problem.

In [ ]:
# upload the custom .cfg back to cloud VM
!cp /mydrive/yolov4/yolov4-obj.cfg ./cfg

## ii) obj.names and obj.data
Create a new file within a code or text editor called **obj.names** where you will have one class name per line in the same order as your classes.txt from the dataset generation step.

In our scenario we have 2 classed, Team 1 and Team 2, because we want to detect players and classify them based on the team.

In [ ]:
# upload the obj.names and obj.data files to cloud VM from Google Drive
!cp /mydrive/yolov4/obj.names ./data
!cp /mydrive/yolov4/obj.data  ./data

## iii) Generating train.txt and test.txt
The last configuration files needed before we can begin to train our custom detector are the train.txt and test.txt files which hold the relative paths to all our training images and valdidation images.

In [ ]:
!python generate_train.py
!python generate_test.py

# Download pre-trained weights
This step downloads the weights for the convolutional layers of the YOLOv4 network. By using these weights it helps our custom object detector to be way more accurate and not have to train as long.

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

# Training the network
Now we are ready to train the network.

In [ ]:
# %%capture
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg yolov4.conv.137 -dont_show -map

In [ ]:
# show chart.png of how custom object detector did with training
imShow('chart.png')

In [ ]:
# kick off training from where it last saved
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights -dont_show -map

# Checking the Mean Average Precision (mAP) of our Model


In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights

# Running detections

In [ ]:
# need to set our custom cfg to test mode
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-obj.cfg
%cd ..

In [ ]:
!python /mydrive/yolov4/yolov4-custom-functions-master/detect.py --weights ./mydrive/yolov4/yolov4-custom-functions-master/data/yolov4-obj_best.weights --size 416 --model yolov4 --images ./mydrive/val/DATASET6386.jpg

In [ ]:
# run the custom detector with this command
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights /mydrive/val/DATASET6386.jpg -thresh 0.01 -ext_output
imShow('predictions.jpg')
!python /mydrive/yolov4/yolov4-custom-functions-master/detect.py --weights ./mydrive/yolov4/yolov4-custom-functions-master/data/yolov4-obj_best.weights --size 416 --model yolov4 --images ./mydrive/val/DATASET6386.jpg

## Running detections on videos

In [ ]:
# Upload the video
upload()

In [ ]:
!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights -dont_show test.mp4 -i 0 -out_filename results.avi

In [ ]:
# download the video with detections shown
download('results.avi')